In [1]:
using NeutronTransport
import Gridap: DiscreteModelFromFile

In [10]:
jsonfile = joinpath("/Users/mariochiaparini/Desktop/Study/json/", "pincell.json")
geometry = DiscreteModelFromFile(jsonfile)

UnstructuredDiscreteModel()

In [11]:

nφ = 32


δ = 1e-2


bcs = BoundaryConditions(top=Reflective, bottom=Reflective, left=Reflective, right=Reflective)

tg = TrackGenerator(geometry, nφ, δ, bcs=bcs)


trace!(tg)

  Number of azimuthal angles in (0, π): 16
  Azimuthal angles in (0, π): [5.71, 16.97, 28.16, 39.44, 50.56, 61.84, 73.03, 84.29, 95.71, 106.97, 118.16, 129.44, 140.56, 151.84, 163.03, 174.29]
  Effective azimuthal spacings: [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
  Total tracks: 3284
  Correct volumes: false

In [12]:

segmentize!(tg)

# polar quadrature
pq = TabuchiYamamoto(6)

# materials
pin = CrossSections("pin", 2;
    νΣf = [1.86278e-2, 3.44137e-1],
    Σt  = [3.62022e-1, 5.72155e-1],
    Σs0 = [3.33748e-1  6.64881e-4; 0.0e-0 3.80898e-1]
)

cladding = CrossSections("cladding", 2;
    Σt  = [2.74144e-1, 2.80890e-1],
    Σs0 = [2.72377e-1 1.90838e-4; 0.0e-0 2.77230e-1]
)

water = CrossSections("water", 2;
    Σt  = [6.40711e-1, 1.69131e-0],
    Σs0 = [6.07382e-1 3.31316e-2; 0.0e-0 1.68428e-0]
)

xs = [pin, cladding, water]


3-element Vector{CrossSections{2, F, Float64, Vector{Float64}, Matrix{Float64}} where F}:
 CrossSections{2, true, Float64, Vector{Float64}, Matrix{Float64}}("pin", [1.0, 0.0], [0.362022, 0.572155], [0.0186278, 0.344137], [0.333748 0.000664881; 0.0 0.380898])
 CrossSections{2, false, Float64, Vector{Float64}, Matrix{Float64}}("cladding", [0.0, 0.0], [0.274144, 0.28089], [0.0, 0.0], [0.272377 0.000190838; 0.0 0.27723])
 CrossSections{2, false, Float64, Vector{Float64}, Matrix{Float64}}("water", [0.0, 0.0], [0.640711, 1.69131], [0.0, 0.0], [0.607382 0.0331316; 0.0 1.68428])

In [13]:
# define the problem
prob = MoCProblem(tg, pq, xs)

# solve
sol = solve(prob, debug=true)

import Gridap: writevtk
import Gridap.Geometry: get_triangulation
trian = get_triangulation(tg.mesh.model)
writevtk(trian, "fluxes-pincell", cellfields=["g1" => sol(1), "g2" => sol(2)])

┌ Info: MoC iterations start...
└ @ NeutronTransport /Users/mariochiaparini/.julia/packages/NeutronTransport/CeajO/src/mocsolver.jl:78
┌ Info: iteration 0
│   sol.keff = 0.5728105655528646
│   ϵ = 0.0
└ @ NeutronTransport /Users/mariochiaparini/.julia/packages/NeutronTransport/CeajO/src/mocsolver.jl:89
┌ Info: iteration 1
│   sol.keff = 0.5806805133010738
│   ϵ = 0.9910092914172143
└ @ NeutronTransport /Users/mariochiaparini/.julia/packages/NeutronTransport/CeajO/src/mocsolver.jl:89
┌ Info: iteration 2
│   sol.keff = 0.5667481960700638
│   ϵ = 0.029051453985711732
└ @ NeutronTransport /Users/mariochiaparini/.julia/packages/NeutronTransport/CeajO/src/mocsolver.jl:89
┌ Info: iteration 3
│   sol.keff = 0.5559436587216016
│   ϵ = 0.04972015926055778
└ @ NeutronTransport /Users/mariochiaparini/.julia/packages/NeutronTransport/CeajO/src/mocsolver.jl:89
┌ Info: iteration 4
│   sol.keff = 0.549190314762393
│   ϵ = 0.0485869023683528
└ @ NeutronTransport /Users/mariochiaparini/.julia/packages/N

(["fluxes-pincell.vtu"],)